<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/Llama_2%E5%8E%9F%E5%A7%8B%E6%A8%A1%E5%9E%8B%E8%BD%AC%E6%8D%A2HF%E6%A0%BC%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

原始模型的转换，这个我对着文档操作过无数次，已经很熟练，这次计划把格式转换的模型，上传到hugingface上，日后对模型是微调，就从自己转换的HF格式，开始进行微调。

另外一个选择，就是直接使用Facebook提供的HF格式。

**重点**

* colab，你需要High-RAM才能完成转换。
* 只需要修改**MODEL_ID** 就可以，支持Llama-2 所有版本。
* colab保存huggingface的token，如果需要上传，需要写入权限的token
* 需要上传转换的模型到hugingface，你需要修改成你的huggingface登录的用户名。
* 7B是13G，13B是26G，70B是140G。进行转换，需要磁盘至少有3倍的剩余空间。

In [26]:
# 模型
MODEL_ID = "meta-llama/Llama-2-13b"
MODEL_NAME = MODEL_ID.split('/')[-1]
NEW_MODEL = MODEL_NAME+'-hf'
Parameter = str.upper(MODEL_NAME.split('-')[2])
username = "chenshake"

In [27]:
print(Parameter)

13B


In [ ]:
# 安装需要依赖
!pip install git+https://github.com/huggingface/transformers
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7
!pip install -q huggingface_hub sentencepiece

In [ ]:
# import 密钥，token
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

In [ ]:
!huggingface-cli download \
	--local-dir=./$MODEL_NAME \
	$MODEL_ID \
  --include "*.json"   "*.model" "*.chk" "*.pth"

In [31]:
# 查看下载结果
import os.path
from pathlib import Path
path=os.path.join(MODEL_NAME)
print(path)
!ls ./$MODEL_NAME

Llama-2-13b
checklist.chk	     consolidated.01.pth  tokenizer_checklist.chk
consolidated.00.pth  params.json	  tokenizer.model


In [32]:
# 格式转换需要提前创建好目录
!mkdir ./$MODEL_NAME/$Parameter
!cp ./$MODEL_NAME/params.json ./$MODEL_NAME/$Parameter/params.json

In [33]:
!ls ./$MODEL_NAME/$Parameter

params.json


In [34]:
# update cache，有时候会出现一个transformers的 cache 的错误提示，提前运行，解决这个问题。
from transformers.utils.hub import move_cache

In [35]:
#解决colab字符集错误，遇到错误再运行也是可以。
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [36]:
# 下载HF格式转换工具
!wget -q https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/llama/convert_llama_weights_to_hf.py

In [37]:
# 开始转换
!python convert_llama_weights_to_hf.py --model_size $Parameter \
    --input_dir ./$MODEL_NAME \
    --output_dir ./$NEW_MODEL

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
2024-01-06 14:50:22.272512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 14:50:22.272663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 14:50:22.449467: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to regis

In [38]:
from huggingface_hub import create_repo, HfApi

# Create empty repo
create_repo(
    repo_id=f"{username}/{NEW_MODEL}",
    repo_type="model",
    exist_ok=True,
)

RepoUrl('https://huggingface.co/chenshake/Llama-2-13b-hf', endpoint='https://huggingface.co', repo_type='model', repo_id='chenshake/Llama-2-13b-hf')

In [39]:
# 把转换HF格式，上传到huggingface,你账号的model里。
api = HfApi()
api.upload_folder(
	folder_path=f"{NEW_MODEL}",
	repo_id=f"{username}/{NEW_MODEL}",
	repo_type="model",
)



pytorch_model-00002-of-00006.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chenshake/Llama-2-13b-hf/commit/37f61ca3c37e7fbb7b0f9c6dc3aa60e843e457bc', commit_message='Upload folder using huggingface_hub', commit_description='', oid='37f61ca3c37e7fbb7b0f9c6dc3aa60e843e457bc', pr_url=None, pr_revision=None, pr_num=None)